In [1]:
import pandas as pd
import numpy as np
from sklearn.impute  import SimpleImputer
pd.set_option('display.max_columns',None)
df=pd.read_csv('/Users/ayush/Desktop/ML-course-digipodium /Preprocessing/train.csv',index_col=0)
df.head()


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
print(df.columns)
print(df.isnull().sum())

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked'],
      dtype='object')
Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64


In [3]:

df.replace('?',np.nan,inplace=True)
df.replace(' ',np.nan,inplace=True)



In [4]:
df.drop('Cabin',axis=1,inplace=True)




In [5]:
missing_cols=df.isnull().sum()
missing_cols[missing_cols > 0 ].index.tolist()

['Age', 'Embarked']

In [6]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Embarked      2
dtype: int64

# pipeline creation


In [7]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [8]:
df.dropna(subset=['Survived'],inplace=True)

In [9]:
X,y=df.drop(columns='Survived'),df['Survived']

In [10]:
num_cols = X.select_dtypes(include=np.number).columns.tolist()
cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
ord_cols = []
hot_encode_cols = []
for col in X[cat_cols]:
    count = X[col].nunique()
    if count <= 2:
        ord_cols.append(col)
    else:
        hot_encode_cols.append(col)
        print(f'{col},{count}')
print(num_cols)
print(ord_cols)
print(hot_encode_cols)

Name,891
Ticket,681
Embarked,3
['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
['Sex']
['Name', 'Ticket', 'Embarked']


In [11]:
num_cols=['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
hot_encode_cols=['Name', 'Ticket', 'Embarked']

In [12]:
num_pipeline=Pipeline([
('imputer',SimpleImputer()),
('scaler',StandardScaler())
])
num_pipeline

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler())])

In [13]:
ord_pipeline=Pipeline([
('imputer',SimpleImputer(strategy='most_frequent')),
('encode',OrdinalEncoder())
])
ord_pipeline

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('encode', OrdinalEncoder())])

In [14]:
OneHotEncoder_pipeline=Pipeline([
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('encode',OneHotEncoder(drop='first',sparse=False)),
])
OneHotEncoder_pipeline

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('encode', OneHotEncoder(drop='first', sparse=False))])

In [15]:
preprocessor=ColumnTransformer([
    ('ord', ord_pipeline, ord_cols),
    ('hot', OneHotEncoder_pipeline, hot_encode_cols),
    ('num', num_pipeline, num_cols),
])
preprocessor

ColumnTransformer(transformers=[('ord',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encode', OrdinalEncoder())]),
                                 ['Sex']),
                                ('hot',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encode',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False))]),
                                 ['Name', 'Ticket', 'Embarked']),
                                ('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'])])

In [16]:
preprocessor.fit_transform(X)
df.head(5)

/Users/ayush/miniconda3/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S
